In [13]:
# May need to install these....
# uncomment if unsure
#!pip install wget
#!pip install dtale

In [1]:
import pandas as pd
import numpy as np
import os

import dtale # data Frame visualization
import wget

# Download the data

Download Current employment statistics from [US Bureau of Labor Statics website](https://www.bls.gov/)


Data Dictionary
???



In [12]:
# Check if file has ben downloaded if not download the file
# This could take a while first time downloading the files (over 5 mins)
# Does this qualify as web scraping? :-)
lnFileName = 'resources/ALLlnSeries.txt'
lnSeriesFileName = 'resources/lnSeries.txt'
lnIndustryFileName = 'resources/lnIndustries.txt'
if not os.path.exists(lnFileName) :
    url = "https://download.bls.gov/pub/time.series/ln/ln.data.1.AllData"
    wget.download(url,lnFileName)
    
    url = "https://download.bls.gov/pub/time.series/ln/ln.series"    
    wget.download(url,lnSeriesFileName)
    
    url = "https://download.bls.gov/pub/time.series/ln/ln.indy"    
    wget.download(url,lnIndustryFileName)  

allLN =  pd.read_table(lnFileName, sep='\t', header=0,names=['series_id', 'year', 'period', 'value', 'footnote_codes'])

LNSeriesDF =  pd.read_table(lnSeriesFileName, sep='\t', header=0,
                            names=['series_id', 'lfst_code', 'periodicity_code', 'series_title','absn_code', 'activity_code', 'ages_code', 'cert_code', 'class_code',
                                   'duration_code', 'education_code', 'entr_code', 'expr_code',
                                   'hheader_code', 'hour_code', 'indy_code', 'jdes_code', 'look_code',
                                   'mari_code', 'mjhs_code', 'occupation_code', 'orig_code', 'pcts_code',
                                   'race_code', 'rjnw_code', 'rnlf_code', 'rwns_code', 'seek_code',
                                   'sexs_code', 'tdat_code', 'vets_code', 'wkst_code', 'born_code',
                                   'chld_code', 'disa_code', 'seasonal', 'footnote_codes', 'begin_year',
                                   'begin_period', 'end_year', 'end_period'],
                            dtype = {'indy_code':np.str_,  'ages_code':np.str_,'education_code':np.str_,'race_code':np.str_}) 





100% [..............................................................................] 14347 / 14347

In [33]:
industriesDF =  pd.read_table(lnIndustryFileName, sep='\t', header=0,dtype={'indy_code':np.str_})    

,indy_code,indy_text
0,0000,All Industries
1,0169,"Agriculture, forestry, fishing, and hunting"
2,0170,Crop production
3,0180,Animal production and aquaculture
4,0190,"Forestry, except logging"


0    0000
1    0000
2    0000
3    0000
4    0000
Name: indy_code, dtype: object

In [26]:
# Only interested in 2020
# CES Seasonally Adjusted
condition = allLN['year'].isin([2020])  & allLN['period'].str.startswith('M')
LNS2020DF = allLN[condition]
LNS2020DF.shape

(59611, 5)

In [27]:
# Get month from period and dop period
# month 13 Annual Average      
# https://download.bls.gov/pub/time.series/ce/ce.period
LNS2020DF['Month'] = LNS2020DF['period'].str.replace('M','').astype(int)
LNS2020DF.drop('period',1,inplace=True)
LNS2020DF.shape

e:\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(59611, 5)

In [28]:
# Merge with Series DF
LNS2020DF = pd.merge(LNS2020DF,LNSeriesDF,on='series_id',how='left')
LNS2020DF.shape


(59611, 45)

In [35]:
LNS2020DF = pd.merge(LNS2020DF,industriesDF,on='indy_code',how='left')
LNS2020DF.shape

(59611, 46)

Exception occurred while processing request: cannot assign without a target object
Traceback (most recent call last):
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\dtale\views.py", line 901, in test_filter
    global_state.get_context_variables(data_id)
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\dtale\utils.py", line 772, in run_query
    df = df.query(final_query, local_dict=context_vars or {})
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py", line 3231, in query
    res = self.eval(expr, **kwargs)
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py", line 3346, in eval
    return _eval(expr, inplace=inplace, **kwargs)
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\computation\eval.py", line 332, in eval
    parsed_expr = Expr(expr, engine=engine, parser=parser, env=env)
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\computation\expr.py", line 764, in __init__
    self.term

2020-05-31 23:24:36,697 - ERROR    - Exception occurred while processing request: cannot assign without a target object
Traceback (most recent call last):
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\dtale\views.py", line 901, in test_filter
    global_state.get_context_variables(data_id)
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\dtale\utils.py", line 772, in run_query
    df = df.query(final_query, local_dict=context_vars or {})
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py", line 3231, in query
    res = self.eval(expr, **kwargs)
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py", line 3346, in eval
    return _eval(expr, inplace=inplace, **kwargs)
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\computation\eval.py", line 332, in eval
    parsed_expr = Expr(expr, engine=engine, parser=parser, env=env)
  File "e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\computation\expr.py"

In [37]:
LNS2020DF.drop(['jdes_code','absn_code','cert_code','rjnw_code', 'begin_year',
                'begin_period', 'end_year', 'end_period'],1,inplace=True)



In [30]:
# Browse the data
# uncomment the following lines to browse the data

#d = dtale.show(LNS2020DF, ignore_duplicate=True)
#d.open_browser()

In [23]:
LNS2020DF.to_pickle("resources/LNS2020.pkl",compression='gzip')

In [24]:
# to load Picke file use the following once pickle has been generated
LNS2020DF_2=pd.read_pickle('resources/LNS2020.pkl',compression='gzip')